In [5]:
import network
import utils
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
net = network.WaterDistributionNetwork("ctown.inp")
net.set_time_params(duration=3600, hydraulic_step=300) # duration=1h, hydstep=5min

In [9]:
status = [1.0, 0.0]
actuators_status_dict = {uid: status for uid in net.pumps.uid.append(net.valves.uid)}

At each timestep the status is chaged from 0 to 1 or viceversa. So, we expect to see the alternation on/off at each new timestep.

In [10]:
net.run(interactive=True, status_dict=actuators_update_dict)

PU1     1.0
PU2     1.0
PU3     1.0
PU4     1.0
PU5     1.0
PU6     1.0
PU7     1.0
PU8     1.0
PU9     1.0
PU10    1.0
PU11    1.0
dtype: float64
PU1     0.0
PU2     0.0
PU3     0.0
PU4     0.0
PU5     0.0
PU6     0.0
PU7     0.0
PU8     0.0
PU9     0.0
PU10    0.0
PU11    0.0
dtype: float64
PU1     1.0
PU2     1.0
PU3     1.0
PU4     1.0
PU5     1.0
PU6     1.0
PU7     1.0
PU8     1.0
PU9     1.0
PU10    1.0
PU11    1.0
dtype: float64
PU1     0.0
PU2     0.0
PU3     0.0
PU4     0.0
PU5     0.0
PU6     0.0
PU7     0.0
PU8     0.0
PU9     0.0
PU10    0.0
PU11    0.0
dtype: float64
PU1     1.0
PU2     1.0
PU3     1.0
PU4     1.0
PU5     1.0
PU6     1.0
PU7     1.0
PU8     1.0
PU9     1.0
PU10    1.0
PU11    1.0
dtype: float64
PU1     0.0
PU2     0.0
PU3     0.0
PU4     0.0
PU5     0.0
PU6     0.0
PU7     0.0
PU8     0.0
PU9     0.0
PU10    0.0
PU11    0.0
dtype: float64
PU1     1.0
PU2     1.0
PU3     1.0
PU4     1.0
PU5     1.0
PU6     1.0
PU7     1.0
PU8     1.0
PU9     1.0
PU10    1.

In [11]:
net.df_links_report.iloc[:, net.df_links_report.columns.get_level_values(2) == 'status']

link        pumps                                                          \
id            PU1    PU2    PU3    PU4    PU5    PU6    PU7    PU8    PU9   
properties status status status status status status status status status   
00:00:00      1.0    1.0    0.0    1.0    0.0    0.0    1.0    1.0    0.0   
00:05:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:10:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    1.0    0.0   
00:11:56      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:15:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    1.0    0.0   
00:20:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:25:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    1.0    0.0   
00:28:36      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:30:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    1.0    0.0   
00:35:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:40:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    1.0    0.0   
00:43:40      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:45:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0   
00:48:50      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:50:00      1.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0   
00:51:20      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:55:00      1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
00:56:46      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
01:00:00      1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

link                     valves                       
id           PU10   PU11     v1    V45    V47     V2  
properties status status status status status status  
00:00:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:05:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:10:00      0.0    0.0    0.0    0.0    0.0    0.0  
00:11:56      1.0    1.0    1.0    1.0    1.0    1.0  
00:15:00      0.0    0.0    0.0    0.0    0.0    1.0  
00:20:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:25:00      0.0    0.0    0.0    0.0    0.0    0.0  
00:28:36      1.0    1.0    1.0    1.0    1.0    1.0  
00:30:00      0.0    0.0    0.0    0.0    0.0    1.0  
00:35:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:40:00      0.0    0.0    0.0    0.0    0.0    0.0  
00:43:40      1.0    1.0    1.0    1.0    1.0    1.0  
00:45:00      0.0    0.0    0.0    0.0    0.0    1.0  
00:48:50      1.0    1.0    1.0    1.0    1.0    1.0  
00:50:00      0.0    0.0    0.0    0.0    0.0    0.0  
00:51:20      1.0    1.0    1.0    1.0    1.0    1.0  
00:55:00      0.0    0.0    0.0    0.0    0.0    0.0  
00:56:46      1.0    1.0    1.0    1.0    1.0    1.0  
01:00:00      0.0    0.0    0.0    0.0    0.0    0.0

Let's check if PU8 respects the control rules of tank T5

In [12]:
net.df_nodes_report['tanks', 'T5', 'pressure']

00:00:00    1.046840
00:05:00    1.117495
00:10:00    1.137751
00:11:56    1.181020
00:15:00    1.232061
00:20:00    1.302462
00:25:00    1.340046
00:28:36    1.359722
00:30:00    1.410591
00:35:00    1.480740
00:40:00    1.518891
00:43:40    1.537562
00:45:00    1.500057
00:48:50    1.516395
00:50:00    1.503350
00:51:20    1.554698
00:55:00    1.537413
00:56:46    1.582669
01:00:00    1.582669
Name: (tanks, T5, pressure), dtype: float64